In [1]:
from scvi.dataset import EbiData, MouseAtlas, UnionDataset, AnnDatasetFromAnnData
from Eval_basis import *
import scanpy as sc
import pandas as pd
import scipy.sparse as sparse
from tqdm import tqdm_notebook as tqdm
import urllib

[2019-09-04 16:33:09,449] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.
/home/michael/anaconda3/envs/scvi/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)


In [2]:
conv = pd.read_csv("./data/gene_maps/hugo_mouse_genes-proteincoding.csv", header=0, index_col=0)
conv.index = conv.index.str.lower()

data_path = os.path.join(("./data"))
mouse_data_path = os.path.join(data_path, "mouse_data")
dsets = [] 
for file in os.listdir(f"{data_path}/mouse_data"):
#     if "droplet" in file:
    dset = sc.read_h5ad(os.path.join(mouse_data_path, file))
    dset.obs.rename(columns={"cell_ontology_class": "cell_types"}, inplace=True)
    
    dset = AnnDatasetFromAnnData(dset)
    
    gns_conved = conv.reindex(np.char.lower(dset.gene_names))["ensembl"]
    if not isinstance(dset.X, np.ndarray):
        X = dset.X.toarray()
    else:
        X = dset.X
    mask = ~gns_conved.isnull()
    
    dset.gene_names = gns_conved[mask].values.astype(str)
    dset.X = X[:, mask]
    dset.cell_types = np.array([ct.replace("ï", "i") for ct in dset.cell_types])
    
    dsets.append(dset)

[2019-09-04 16:33:12,297] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 16:33:12,840] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:33:12,840] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:33:13,011] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:33:13,192] INFO - scvi.dataset.dataset | Downsampled from 7916 to 7916 cells
[2019-09-04 16:33:14,176] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:33:15,222] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 16:33:15,530] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:33:15,531] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:33:15,607] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:33:15,708] IN

[2019-09-04 16:34:22,643] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 16:34:22,886] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:34:22,887] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:34:22,934] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:34:22,976] INFO - scvi.dataset.dataset | Downsampled from 1561 to 1561 cells
[2019-09-04 16:34:23,144] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:34:23,855] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 16:34:24,168] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:34:24,169] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:34:24,238] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:34:24,313] IN

[2019-09-04 16:34:59,381] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:34:59,618] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:34:59,864] INFO - scvi.dataset.dataset | Downsampled from 9734 to 9734 cells
[2019-09-04 16:35:01,261] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:35:04,847] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:35:04,848] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:35:05,228] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:35:05,640] INFO - scvi.dataset.dataset | Downsampled from 29322 to 29322 cells
[2019-09-04 16:35:10,178] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:35:14,746] INFO - scvi.dataset.anndataset | Dense size under 1Gb, casting to dense format (np.ndarray).
[2019-09-04 16:35:15,196] INFO - scvi.data

In [3]:
mouse_muris_senis = UnionDataset("./data", 
                                 gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding", 
                                 low_memory=False)
mouse_muris_senis.join_datasets(data_source="memory", 
                                data_target="memory",
                                gene_datasets=dsets)
mouse_muris_senis.name = "Tabula Muris Senis"

Concatenating datasets: 100%|██████████| 38/38 [05:01<00:00,  6.51s/it]
[2019-09-04 16:40:30,992] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 16:40:37,060] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:40:37,066] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 16:40:37,070] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:40:37,104] INFO - scvi.dataset.dataset | Joined 38 datasets to one of shape 290905 x 22250.


In [4]:
dsets = None
mouse_muris_senis.cell_types = np.array([ct.replace("ï", "i") for ct in mouse_muris_senis.cell_types])

In [5]:
def do_procedure(dataset, other_data, model_filename, tsne_filename):
    n_epochs = 100
    colors = None

    print("Training VAE")

    trainer = train_vae(dataset, "./data", f"../trained_models/{model_filename}", n_epochs=n_epochs)

    dot_size = (mpl.rcParams['lines.markersize'] ** 2.0)

    _ = plot_tsne(trainer, trainer.model, dataset, f"./plots/{tsne_filename}", image_datatype="pdf",
                  colors=colors, s=dot_size, edgecolors='black')

    _ = plot_tsne(trainer, trainer.model, other_data,
                  f"./plots/test_data_in_{tsne_filename}",
                  image_datatype="pdf", colors=colors, s=dot_size, edgecolors='black')

In [6]:
ebi = UnionDataset("./data",
                   gene_map_load_filename="gene_maps/ensembl_mouse_genes-proteincoding",
                   low_memory=False)
ebi.join_datasets(data_source="memory",
                  data_target="memory",
                  gene_datasets=[EbiData("./data")])

[2019-09-04 16:41:23,923] WARNING - scvi.dataset.dataset | This dataset has some empty cells, this might fail scVI inference.Data should be filtered with `my_dataset.filter_cells_by_count()
[2019-09-04 16:41:24,010] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:41:24,012] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2019-09-04 16:41:25,131] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2019-09-04 16:41:26,254] INFO - scvi.dataset.dataset | Downsampled from 50896 to 35577 cells
Concatenating datasets: 100%|██████████| 1/1 [00:49<00:00, 49.76s/it]
[2019-09-04 16:42:16,563] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 16:42:17,734] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2019-09-04 16:42:17,735] INFO - scvi.dataset.dataset | Union dataset is set to ignore batch annotation.
[2019-09-04 16:42:17,736] INFO - scvi.dataset.dataset | Remapping labels to 

GeneExpressionDataset object with n_cells x nb_genes = 35577 x 22250
    gene_attribute_names: 'gene_names'
    cell_attribute_names: 'local_vars', 'batch_indices', 'labels', 'local_means'
    cell_categorical_attribute_names: 'batch_indices', 'labels'

In [7]:
do_procedure(mouse_muris_senis, ebi, "mouse_muris_senis", "mouse_muris_senis_tsne")

Training VAE
Initializing training.
training: 100%|██████████| 100/100 [1:01:06<00:00, 43.90s/it]
